In [ ]:
import os
import duckdb
from dotenv import load_dotenv
import keplergl

In [ ]:
load_dotenv()
connection_string = os.getenv("BLOB_STORAGE_CONNECTION_STRING")
print("Connection string:", connection_string)

con = duckdb.connect(":memory:")
con.execute("INSTALL spatial; LOAD spatial;")
con.execute("INSTALL azure; LOAD azure;")
con.execute("SET azure_storage_connection_string = ?;", [os.getenv("BLOB_STORAGE_CONNECTION_STRING")])
con.execute("SET azure_transport_option_type = curl")

# Verify that connection string is set in duckdb

In [ ]:
con.sql("""SELECT current_setting('azure_storage_connection_string')""")

In [ ]:
con.sql("""
    SELECT * FROM parquet_schema('az://raw/release/2025-11-06.3/dataset=fkb/theme=buildings/region=18/**.parquet')
""")